# Imports and Constants

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import sklearn.linear_model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import random

from savingOutputs import *
from loadData import *
from masks import *
from decoding import *
from plots import *
from utility import *
from metrics import *

In [ ]:
from_who = "mohameds"   # 2 possibilities : "mohameds" or "our"
maps_folder="brain_maps/"+from_who+"_maps"
masks_folder="masks/"+from_who+"_masks"

SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
labels = {'vis' : np.array(classes*nb_runs), 'aud' : np.array(classes*nb_runs)}
labels_same = np.array(classes*nb_runs)
subjects_ids = range(1, 24)
n_subjects = len(subjects_ids)
n_individual_perms = 1000

within_modal_tasks_regions = [(["vis"], ["V5_L", "V5_R"]),
                           (["vis"], ["PT_L", "PT_R"]),
                           (["aud"], ["V5_L", "V5_R"]),
                           (["aud"], ["PT_L", "PT_R"])]

cross_modal_tasks_regions = [(["vis", "aud"], ["V5_L", "V5_R"]),
                            (["vis", "aud"], ["PT_L", "PT_R"])]

std_scaler = sklearn.preprocessing.StandardScaler()

classifiers = {
    'svm':sklearn.svm.SVC(C=1, random_state=SEED),
    'LR':sklearn.linear_model.LogisticRegression()}

param_grids = {
    'svm':{
        #'svm__C': [0.1, 1], 
        #'svm__gamma': [1, 0.1],
        'svm__kernel': ['linear']},  
    'LR':{
        'LR__C': [1]}}

models = dict()
for name in classifiers:
    pipeline = Pipeline([('scaler', std_scaler), (name, classifiers[name])])
    GS = GridSearchCV(pipeline, param_grids[name], cv = nb_runs - 1)
    models[name] = GS
    
decoder = Decoder(n_perm=n_individual_perms, models=models, n_classes=len(classes), n_splits=nb_runs, seed=SEED)

# Loading data

In [ ]:
maps_masked, masks_exist = load_full_data(subjects_ids, len(classes), nb_runs, maps_folder, masks_folder, is_from_mohamed=(from_who=="mohameds"))
decoder.set_masks_exist(masks_exist)

# Within-modality decoding

In [ ]:
confusion_matrixes_within = decoder.within_modality_decoding(maps_masked, labels, subjects_ids, within_modal_tasks_regions)
confusion_matrixes_within = change_confusion_matrixes_org(confusion_matrixes_within, subjects_ids, models.keys())

# Cross-modal decoding

In [ ]:
confusion_matrixes_cross = decoder.cross_modality_decoding(maps_masked, labels, subjects_ids, cross_modal_tasks_regions)
confusion_matrixes_cross = change_confusion_matrixes_org(confusion_matrixes_cross, subjects_ids, models.keys())

# Bootstrapping to assess group-level significance

In [ ]:
n_single_perm = 100
n_bootstrap = 100_000

In [ ]:
within_cf_100_perm = decoder.score_bootstrapped_permutations(n_single_perm, labels_same, within_modal_tasks_regions,maps_masked,n_subjects, within_modality=True)
within_scores_100_perm = compute_accuracy_bootstrap(n_subjects, n_single_perm, within_cf_100_perm, len(classes))
bootstrapped_distribution_within = compute_bootstrap_distribution(n_bootstrap, n_subjects, within_scores_100_perm, n_single_perm)

In [ ]:
cross_cfm_100_perm = decoder.score_bootstrapped_permutations(n_single_perm, labels_same, cross_modal_tasks_regions,maps_masked,n_subjects, within_modality=False)
cross_scores_100_perm = compute_accuracy_bootstrap(n_subjects, n_single_perm, cross_cfm_100_perm, len(classes))
bootstrapped_distribution_cross = compute_bootstrap_distribution(n_bootstrap, n_subjects, cross_scores_100_perm, n_single_perm)

# Saving results

In [ ]:
out_directory = "out/"+from_who+"_"
for name in classifiers:
    out_dir = out_directory+str(classifiers[name])+"/"
    create_directory(out_dir)
    
    save_dicts(out_dir+"confusion_matrixes_within.csv", confusion_matrixes_within[name], list(confusion_matrixes_within[name][0].keys()), subjects_ids)
    acc_within = compute_metric(out_dir, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "within", masks_exist, len(classes), ret = True)
    save_dicts(out_dir+"masks_exist.csv", masks_exist, list(masks_exist[0].keys()), subjects_ids)

    compute_metric(out_dir, subjects_ids, {'name' : 'recall', 'function':recall}, "within", masks_exist, len(classes))
    compute_metric(out_dir, subjects_ids, {'name' : 'precision', 'function':precision}, "within", masks_exist, len(classes))

    save_dicts(out_dir+"confusion_matrixes_cross.csv", confusion_matrixes_cross[name], list(confusion_matrixes_cross[name][0].keys()), subjects_ids)
    acc_cross = compute_metric(out_dir, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "cross", masks_exist, len(classes), ret = True)
    
    compute_metric(out_dir, subjects_ids, {'name' : 'recall', 'function':recall}, "cross", masks_exist, len(classes))
    compute_metric(out_dir, subjects_ids, {'name' : 'precision', 'function':precision}, "cross", masks_exist, len(classes))

    within_modality_group_results = average_dicos(acc_within)
    save_dicts(out_dir+"group_scores_within.csv", [within_modality_group_results], list(within_modality_group_results.keys()), [0])

    cross_modality_group_results = average_dicos(acc_cross)
    save_dicts(out_dir+"group_scores_cross.csv", [cross_modality_group_results], list(cross_modality_group_results.keys()), [0])

In [ ]:
save_dicts(out_directory+"accuracy_bootstraps_within.csv", [bootstrapped_distribution_within], list(bootstrapped_distribution_within[0].keys()), range(n_bootstrap))
save_dicts(out_directory+"accuracy_bootstraps_cross.csv", [bootstrapped_distribution_cross], list(bootstrapped_distribution_cross[0].keys()), range(n_bootstrap))

# Plotting results (from files of saved results)

In [ ]:
for name in classifiers:
    out_dir = out_directory+str(classifiers[name])+"/"
    cv_group_df = retrieve_cv_metric(out_dir, "group_scores")
    cv_df = retrieve_cv_metric(out_dir, "accuracy")
    cfm_df = retrieve_cv_matrixes(out_dir)

    plt_dir = "plots/"+from_who+"_"+str(classifiers[name])
    create_directory(plt_dir)
    plotter = Plotter(plt_dir, subjects_ids)
    plotter.plot_cv_score_with_points(cv_df, chance_level = True)

    group_cfm = compute_group_confusion_matrix(cfm_df, subjects_ids)
    plotter.plot_group_confusion_matrix(group_cfm, classes)

In [ ]:
# compute estimated p-values for bootstrap
bootstrap_df  = retrieve_bootstrap_metric(results_directory, "accuracy")
pvals = compute_p_val_bootstrap(bootstrap_df, cv_group_df)
save_dicts(out_directory+"estimated_pval_bootstrap.csv", [pvals], list(pvals.keys()), [0])

In [ ]:
plotter.plot_bootstrap(bootstrap_df, cv_group_df, pvals, 30)